# Height-Balanced Trees: From Theory to Lab
 

## 1. Quick BST Refresher 

**Objective**: Quick BST refresher, then understand why we need balanced trees

### **What is a Binary Search Tree?**

A BST is a tree data structure where **each node has at most two children** with a special ordering property:

```python
# BST Property:
# - Left child < Parent
# - Right child > Parent  
# - Both subtrees are also BSTs
```

### **BST as Dictionaries**

We can represent BST nodes using dictionaries (which you already know!):

```python
# A BST node is just a dictionary!
node = {
    'value': 10,
    'left': None,    # Another node dictionary for smaller values
    'right': None    # Another node dictionary for larger values
}
```

### **BST Operations - The Recursive Pattern**

**Insertion** follows a beautiful recursive pattern:

```python
def bst_insert(root, value):
    # Base case: empty spot found
    if root is None:
        return {'value': value, 'left': None, 'right': None}
    
    # Recursive case: go left or right
    if value < root['value']:
        root['left'] = bst_insert(root['left'], value)
    else:
        root['right'] = bst_insert(root['right'], value)
    
    return root
```

**Search** uses the same recursive thinking:

```python
def bst_search(root, target):
    # Base case 1: not found
    if root is None:
        return None
    
    # Base case 2: found!
    if root['value'] == target:
        return root
    
    # Recursive case: search left or right
    if target < root['value']:
        return bst_search(root['left'], target)
    else:
        return bst_search(root['right'], target)
```

### **Visual Example**

Let's build a BST step by step:

```python
# Insert: 50, 30, 70, 20, 40
root = None
root = bst_insert(root, 50)  # Root: 50
root = bst_insert(root, 30)  # 30 goes left of 50
root = bst_insert(root, 70)  # 70 goes right of 50  
root = bst_insert(root, 20)  # 20 goes left of 30
root = bst_insert(root, 40)  # 40 goes right of 30

# Result:
#       50
#      /  \
#     30   70
#    /  \
#   20   40
```

**Performance**: In a well-shaped BST, operations take O(h) time where h is the tree height.

---

## ⚠️ **The BST Problem**

### **The Hidden Danger: Sorted Input**

What happens when we insert sorted data?

```python
# Insert sorted sequence - DISASTER!
root = None
for value in [10, 20, 30, 40, 50]:
    root = bst_insert(root, value)

# Result:
# 10
#   \
#    20
#      \
#       30
#         \
#          40
#            \
#             50
```

**This isn't a tree anymore - it's a linked list!**

### **Performance Impact**

Let's compare the shapes:

**Balanced BST** vs **Unbalanced BST**

```
Balanced:          Unbalanced:
     30                10
   /    \                \
  20     40               20
 /      /  \                \
10     35   50               30
                                \
                                 40
                                    \
                                     50
```

**Operation Costs**:
- **Balanced**: Height ≈ log₂(n) → O(log n) operations
- **Unbalanced**: Height = n → O(n) operations

### **Real-World Consequences**

```python
# Imagine a user database
user_ids = [1001, 1002, 1003, ..., 1000000]  # Sorted user IDs

# Build BST - becomes a linked list!
user_tree = None
for user_id in user_ids:
    user_tree = bst_insert(user_tree, user_id)

# Searching for user 1000000 takes 1,000,000 steps!
# This could make your application feel SLOW
```

### **The Root Cause**

The problem: **Regular BSTs don't care about their shape** - they only maintain the ordering property.

We need trees that **actively maintain good shape** for performance!

---

## ⚖️ **The Balance Solution**

### **Height-Balanced Trees to the Rescue!**

A **height-balanced tree** maintains this property:

> For every node, the heights of left and right subtrees differ by at most 1.

### **Balance Factor**

We measure balance using a simple calculation:

```python
balance_factor = height(left_subtree) - height(right_subtree)
```

**Valid values**: -1, 0, or +1

### **Visual Examples**

**Balanced Example**:
```
          A [bf=0]
         / \
[bf=+1] B   C [bf=-1]
       /     \
      D       E

Node A: 1 - 1 = 0 ✓
Node B: 1 - 0 = 1 ✓  
Node C: 0 - 1 = -1 ✓
```

**Unbalanced Example**:
```
        A [bf=+2]  ← PROBLEM!
       /
      B [bf=+1]
     /
    C

Node A: 2 - 0 = 2 ✗
```

### **The Performance Guarantee**

By maintaining balance, we get:

| Operation | Balanced Tree | Unbalanced Tree |
|-----------|---------------|-----------------|
| **Search** | O(log n) | O(n) |
| **Insert** | O(log n) | O(n) |
| **Delete** | O(log n) | O(n) |

**For 1,000,000 elements**:
- **Balanced**: ~20 operations (log₂(1M) ≈ 20)
- **Unbalanced**: up to 1,000,000 operations

### **Self-Balancing Trees**

The magic: **Trees that balance themselves automatically!**

- **AVL Trees**: Our focus - use height differences
- **Red-Black Trees**: Use color coding
- **B-Trees**: For disk-based systems

**Key Insight**: We'll implement AVL trees using the same dictionary and recursion patterns you already know!

---

## 🎯 **Key Takeaways**

1. **BST Review**: Trees with left < parent < right, built with dictionaries & recursion
2. **Problem**: BSTs can become unbalanced with sorted input → O(n) performance
3. **Solution**: Maintain height balance at every node  
4. **Benefit**: Guaranteed O(log n) operations
5. **Approach**: Self-balancing trees that automatically maintain balance

### **Bridge to Next Phase**

> "Now that we see the problem and the solution concept, let's dive into HOW AVL trees maintain balance through intelligent rotations..."

---

## 📝 **Student Checkpoint Questions**

1. **Quick Think**: What's the recursive base case for BST search?
2. **Visual Analysis**: Is this BST balanced? Why/why not?
   ```
       50
      /  \
     30   70
    /    /  \
   20   60   80
  /
 10
   ```

# 2. AVL Tree Concepts & Rotation Mechanisms
 
**Objective**: Understand AVL tree properties and the four rotation cases that maintain balance

---

## 🏗️ **AVL Tree Properties**

### **What is an AVL Tree?**

An **AVL tree** (named after inventors Adelson-Velsky and Landis) is a self-balancing Binary Search Tree that maintains its balance through automatic rotations.

**Key Properties**:
1. **BST Property**: Maintains the binary search tree ordering (left < parent < right)
2. **Balance Property**: For every node, |height(left) - height(right)| ≤ 1
3. **Self-Balancing**: Automatically rebalances after insertions and deletions

### **The Balance Factor Revisited**

```python
# For any node, we calculate:
balance_factor = height(left_subtree) - height(right_subtree)

# Valid values: -1, 0, or +1
# If |balance_factor| > 1 → need to rebalance!
```

### **AVL Tree vs Regular BST**

**Regular BST**:
```python
# Can become unbalanced
# No guarantee of performance
# Simple but risky
```

**AVL Tree**:
```python
# Always maintains balance  
# Guaranteed O(log n) performance
# Slightly more complex but reliable
```

### **Visual AVL Example**

```
       44 [h=3, bf=0]
      /    \
    17      62 [h=2, bf=0]
   /  \     /  \
  8    32  50   78 [h=1, bf=-1]
      /      \    \
     28       54   88
     
Every node has balance factor in {-1, 0, +1} ✓
```

---

## 🔄 **Rotation Mechanisms**

### **The Four Rotation Cases**

When a node becomes unbalanced (|balance_factor| = 2), we perform rotations to restore balance. There are four cases:

### **Case 1: Left-Left (LL) - Right Rotation**

**Scenario**: Node is left-heavy, and left child is left-heavy or balanced

```
Before:
    A (bf=+2)  ← Unbalanced!
   /
  B (bf=+1) 
 /
C

Right Rotation on A:
  B
 / \
C   A
```

**When**: Insertion into left subtree of left child

### **Case 2: Right-Right (RR) - Left Rotation**  

**Scenario**: Node is right-heavy, and right child is right-heavy or balanced

```
Before:
A (bf=-2)  ← Unbalanced!
 \
  B (bf=-1)
   \
    C

Left Rotation on A:
  B
 / \
A   C
```

**When**: Insertion into right subtree of right child

### **Case 3: Left-Right (LR) - Left-Right Rotation**

**Scenario**: Node is left-heavy, but left child is right-heavy

```
Before:
    A (bf=+2)  ← Unbalanced!
   /
  B (bf=-1) 
   \
    C

Step 1: Left Rotation on B
    A
   /
  C
 /
B

Step 2: Right Rotation on A
    C
   / \
  B   A
```

**When**: Insertion into right subtree of left child

### **Case 4: Right-Left (RL) - Right-Left Rotation**

**Scenario**: Node is right-heavy, but right child is left-heavy

```
Before:
A (bf=-2)  ← Unbalanced!
 \
  C (bf=+1)
 /
B

Step 1: Right Rotation on C
A
 \
  B
   \
    C

Step 2: Left Rotation on A
  B
 / \
A   C
```

**When**: Insertion into left subtree of right child

---

## 🎯 **Rotation Function Signatures**

Here are the function signatures we'll implement:

### **Core Rotation Functions**

```python
def rotate_left(unbalanced_node):
    """
    Input: Node dictionary that is right-heavy
    Returns: New root node after left rotation
    Properties: Modifies tree structure, maintains BST property
    Example: Converts A → B → C into ..
    """
    pass

def rotate_right(unbalanced_node):
    """
    Input: Node dictionary that is left-heavy  
    Returns: New root node after right rotation
    Properties: Modifies tree structure, maintains BST property
    Example: Converts C ← B ← A into ..
    """
    pass
```

### **Balance  Function**

```python
def balance_node(node):
    """
    Input: Node dictionary that might be unbalanced
    Returns: Balanced node (possibly new root after rotations)
    Properties: Detects imbalance and applies correct rotation
    Handles all 4 cases: LL, RR, LR, RL
    """
    pass
```

---

## 🔍 **Visual Examples of Each Case**

### **LL Case Example**
```python
# Insert sequence: 30, 20, 10
# Before rotation:
#     30 (bf=+2)
#    /
#   20 (bf=+1) 
#  /
# 10

# After right rotation on 30:
#   20
#  /  \
# 10   30
```

### **RR Case Example**  
```python
# Insert sequence: 10, 20, 30
# Before rotation:
# 10 (bf=-2)
#   \
#    20 (bf=-1)
#      \
#       30

# After left rotation on 10:
#    20
#   /  \
#  10   30
```

### **LR Case Example**
```python
# Insert sequence: 30, 10, 20
# Before rotation:
#     30 (bf=+2)
#    /
#   10 (bf=-1)
#     \
#      20

# After left-right rotation:
#    20
#   /  \
#  10   30
```

### **RL Case Example**
```python
# Insert sequence: 10, 30, 20
# Before rotation:
# 10 (bf=-2)
#   \
#    30 (bf=+1)
#   /
#  20

# After right-left rotation:
#    20
#   /  \
#  10   30
```

---

## 💡 **Key Insights About Rotations**

### **What Rotations Preserve**
1. **BST Property**: Ordering remains correct
2. **Inorder Sequence**: The sorted order doesn't change
3. **Tree Contents**: All values remain in the tree

### **What Rotations Change**  
1. **Tree Height**: Reduces overall height
2. **Balance Factors**: Restores balance
3. **Performance**: Maintains O(log n) operations

### **The Magic Formula**
```python
# After every insertion:
1. Insert like normal BST (recursively)
2. Update heights (bottom-up)
3. Check balance factor  
4. If unbalanced, apply appropriate rotation
5. Return the (possibly new) root
```

---

## 🎯 **Key Takeaways**

1. **AVL Trees** maintain strict height balance through automatic rotations
2. **Four Imbalance Cases**: LL, RR, LR, RL
3. **Rotation Types**: Single rotations (left/right) and double rotations
4. **Rotations Preserve**: BST property and inorder sequence
5. **Rotations Improve**: Tree height and performance guarantees

### **Bridge to Next Phase -- Today's Lab Session**

> "Now that we understand WHEN and WHY we need rotations, let's see HOW we implement this using the dictionary and recursion patterns we know..."

---

## 📝 **Student Checkpoint Questions**

1. **Quick Think**: What type of rotation is needed for this tree?
   ```
       50 (bf=+2)
      /
     30 (bf=-1) 
      \
       40
   ```

2. **Visual Analysis**: After a right rotation, what happens to the balance factors?

3. **Why Double Rotations?**: Why can't we use single rotations for LR and RL cases?

---

# 3. Function Signatures
 
**Objective**: Design the dictionary-based node structure and define all function signatures for AVL tree implementation

---

## 🏗️ **Dictionary Representation**


### **The AVL Node Dictionary Structure**

```python
# Each node is a dictionary with 4 keys:
node = {
    'value': 42,           # The actual data
    'left': None,          # Another node dictionary (smaller values)
    'right': None,         # Another node dictionary (larger values) 
    'height': 1            # Critical for balancing decisions
}
```

## 📋 **Core Function Signatures**

Here are all the functions we'll need, with clear specifications:

### **1. Basic Node Operations**

```python
def create_node(value):
    """
    Creates and returns a new AVL node dictionary.
    
    Parameters:
        value: The value to store in the node
        
    Returns:
        A dictionary: {'value': value, 'left': None, 'right': None, 'height': 1}
        
    Properties:
        - Pure function (no side effects)
        - Always returns a new structure
        - Sets initial height to 1 (leaf node)
    """
    pass
```

### **2. Height and Balance Calculations**

```python
def get_height(node):
    """
    Returns the height of a node.
    
    Parameters:
        node: A node dictionary or None
        
    Returns:
        Integer height (0 if node is None)
        
    Properties:
        - Recursive function
        - Handles the None base case gracefully
        - Pure function (no modifications)
    """
    pass

def get_balance(node):
    """
    Calculates the balance factor of a node.
    
    Parameters:
        node: A node dictionary or None
        
    Returns:
        Integer: height(left) - height(right)
        
    Properties:
        - Uses get_height() internally
        - Returns 0 for None nodes
        - Pure function
    """
    pass

def update_height(node):
    """
    Updates the height of a node based on its children.
    
    Parameters:
        node: The node dictionary to update
        
    Returns:
        None (modifies the node in place)
        
    Properties:
        - Modifies the input dictionary
        - Uses get_height() on children
        - Called after structural changes
    """
    pass
```

### **3. Rotation Functions**

```python
def rotate_left(unbalanced_node):
    """
    Performs a left rotation on an unbalanced node.
    
    Parameters:
        unbalanced_node: The root of the unbalanced subtree
        
    Returns:
        The new root node after rotation
        
    Properties:
        - Modifies the tree structure
        - Updates heights of affected nodes
        - Maintains BST property
        - Used for RR and RL cases
    """
    pass

def rotate_right(unbalanced_node):
    """
    Performs a right rotation on an unbalanced node.
    
    Parameters:
        unbalanced_node: The root of the unbalanced subtree
        
    Returns:
        The new root node after rotation
        
    Properties:
        - Modifies the tree structure  
        - Updates heights of affected nodes
        - Maintains BST property
        - Used for LL and LR cases
    """
    pass
```

### **4. Core AVL Operations**

```python
def insert_node(root, value):
    """
    Inserts a value into the AVL tree and maintains balance.
    
    Parameters:
        root: The root node of the (sub)tree
        value: The value to insert
        
    Returns:
        The new root of the (sub)tree after insertion and balancing
        
    Properties:
        - Recursive function
        - Returns modified structure (functional style)
        - Handles all 4 rotation cases
        - Time complexity: O(log n)
    """
    pass

def search_node(root, value):
    """
    Searches for a value in the AVL tree.
    
    Parameters:
        root: The root node of the (sub)tree to search
        value: The value to find
        
    Returns:
        The node dictionary if found, None otherwise
        
    Properties:
        - Recursive function  
        - Pure function (no modifications)
        - Time complexity: O(log n)
        - Same algorithm as BST search
    """
    pass
```

### **5. Utility Functions**

```python
def display_tree(root, level=0, prefix="Root: "):
    """
    Displays the tree structure with balance information.
    
    Parameters:
        root: The root node to display
        level: Current depth (for indentation)
        prefix: Label for the current node
        
    Returns:
        None (prints to console)
        
    Properties:
        - Recursive function
        - Shows values, heights, and balance factors
        - Great for debugging and visualization
    """
    pass

def inorder_traversal(root, result=None):
    """
    Returns the sorted values from the tree.
    
    Parameters:
        root: The root node to traverse
        result: List to accumulate results (internal use)
        
    Returns:
        List of values in sorted order
        
    Properties:
        - Recursive function
        - Pure function (no modifications)
        - Returns the BST elements in sorted order
    """
    pass
```

---

## 🚀 **Complete Function Signature Summary**

```python
# ========================
# AVL TREE FUNCTION LIBRARY
# ========================

# Node Management
create_node(value)          # → new node dict
get_height(node)            # → integer height  
get_balance(node)           # → integer balance factor
update_height(node)         # → None (modifies)

# Rotations
rotate_left(node)           # → new root node
rotate_right(node)          # → new root node

# Core Operations  
insert_node(root, value)    # → new root node
search_node(root, value)    # → node or None

# Utilities
display_tree(root)          # → None (prints)
inorder_traversal(root)     # → sorted list
```

---

## 🎯 **Key Takeaways from Phase 3**

1. **Dictionary Representation**: Each node is a dict with value, left, right, height
2. **Function Design**: Clear signatures with specified inputs, outputs, and properties  
3. **Recursive Patterns**: Base cases + recursive cases + result combination
4. **Separation of Concerns**: Different functions for different responsibilities
5. **Modular Design**: Each function does one thing well

### **Bridge to Next Phase**

> "With our function signatures defined, we're ready to see these concepts in action through live demonstration and understand how they work together..."

---

## 📝 **Student Checkpoint Questions**

1. **Function Analysis**: Which functions modify the tree structure vs which are read-only?
2. **Recursive Thinking**: Why does `insert_node` return the root instead of modifying it in place?
3. **Design Choice**: Why do we use `update_height` as a separate function instead of calculating height on demand?

---

**Next**: Live Demonstration in the Lab Sessions